In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
    self.conv1_bn = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
    self.conv2_bn = nn.BatchNorm2d(16)
    self.drop1 = nn.Dropout(p=0.1)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
    self.conv3_bn = nn.BatchNorm2d(16)
    self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
    self.conv4_bn = nn.BatchNorm2d(16)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.conv5 = nn.Conv2d(16, 16, 3)
    self.conv5_bn = nn.BatchNorm2d(16)
    self.drop5 = nn.Dropout(p=0.1)
    self.conv6 = nn.Conv2d(16, 32, 3)
    self.conv6_bn = nn.BatchNorm2d(32)
    self.conv7 = nn.Conv2d(32, 10, 3)

  def forward(self, x):
    
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv1_bn(x)
    
    x = self.conv2(x)
    x = F.relu(x)
    x = self.conv2_bn(x)

    x = self.drop1(x)

    x = self.pool1(x)
    
    x = self.conv3(x)
    x = F.relu(x)
    x = self.conv3_bn(x)
    
    x = self.conv4(x)
    x = F.relu(x)
    x = self.conv4_bn(x)

    x = self.pool2(x)
    
    x = self.conv5(x)
    x = F.relu(x)
    x = self.conv5_bn(x)
    
    x = self.drop1(x)

    x = self.conv6(x)
    x = F.relu(x)
    x = self.conv6_bn(x)
    
    x = self.conv7(x)
    x = x.view(-1, 10)
    return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
       BatchNorm2d-4           [-1, 16, 28, 28]              32
           Dropout-5           [-1, 16, 28, 28]               0
         MaxPool2d-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           2,320
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 16, 14, 14]           2,320
      BatchNorm2d-10           [-1, 16, 14, 14]              32
        MaxPool2d-11             [-1, 16, 7, 7]               0
           Conv2d-12             [-1, 16, 5, 5]           2,320
      BatchNorm2d-13             [-1, 16, 5, 5]              32
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
def train_metric(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)

    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    train_metric(model, device, train_loader)
    test(model, device, test_loader)

loss=0.19285258650779724 batch_id=47:  10%|█         | 47/469 [00:02<00:19, 22.06it/s]

loss=0.1402018815279007 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.51it/s]



Train set: Average loss: 0.0437, Accuracy: 59202/60000 (98.67%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0413, Accuracy: 9861/10000 (98.61%)



...


Train set: Average loss: 0.0042, Accuracy: 59936/60000 (99.89%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0217, Accuracy: 9936/10000 (99.36%)



loss=0.008046763017773628 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.49it/s]



Train set: Average loss: 0.0025, Accuracy: 59971/60000 (99.95%)


Test set: Average loss: 0.0192, Accuracy: 9940/10000 (99.40%)

